In [69]:
import pickle
import matplotlib.pyplot as plt
import numpy as np
from Config import HYPERPARAETERS
import pandas as pd

#Constants
c = 10
d = 400
k = 10
R0 = 1000

In [70]:
DATA_COLUMNS = ['league', 'season', 'week', 'home_team', 'away_team', 'result']
dataset: pd.DataFrame = pd.read_csv(
            'data/KaggleDataset_withBO.csv',
            encoding='utf-8',
            usecols=DATA_COLUMNS,
            dtype=dict(zip(DATA_COLUMNS, [str]*2 + [int] + [str]*3))
        )
conds = [
    (dataset['result'] == 'win'),
    (dataset['result'] == 'tie'),
    (dataset['result'] == 'loss')
]
choices = [1, 0.5, 0]
dataset['alphaH'] = np.select(conds, choices)
dataset['alphaA'] = np.select(conds, choices[::-1])

Elo_Scores = pd.Series(R0, index=np.unique(dataset[['home_team', 'away_team']]))


tvt_indcs = dict() #train validation test indcs
for league, league_df in dataset.groupby('league'):
    tvt_indcs[league] = dict()
    tvt_indcs[league]['test'] = league_df.index[-int(HYPERPARAETERS.TestPortion.value * league_df.shape[0]):]
    tvt_indcs[league]['eval'] = league_df.index[-int((HYPERPARAETERS.TestPortion.value + HYPERPARAETERS.ValidationPortion.value) * league_df.shape[0]) : -int(HYPERPARAETERS.TestPortion.value * league_df.shape[0])]
    tvt_indcs[league]['train'] = league_df.index[: -int((HYPERPARAETERS.TestPortion.value + HYPERPARAETERS.ValidationPortion.value) * league_df.shape[0])]

In [71]:
for league, league_df in dataset.groupby('league'):
    index_dict = tvt_indcs[league]
    for idx in index_dict['train']:
        match = dataset.loc[idx, :]
        currentH_elo = Elo_Scores[match['home_team']]
        currentA_elo = Elo_Scores[match['away_team']]
        expectedH = 1 / (1 + c**((currentA_elo - currentH_elo) / d))
        expectedH = 1 / (1 + c**((currentH_elo - currentA_elo) / d))
        
        #Updating
        Elo_Scores.loc[match['home_team']] = currentH_elo + (k * (match['alphaH'] - expectedH))
        Elo_Scores.loc[match['home_team']] = currentH_elo + (k * (match['alphaA'] - expectedH))

In [72]:
Elo_Scores

AC Ajaccio                  1118.179993
AC Arles-Avignon            1048.378342
ADO Den Haag                1148.760219
AJ Auxerre                   947.687251
AS Monaco                    925.685929
                               ...     
Wigan Athletic              1127.055064
Willem II                   1163.025110
Wolverhampton Wanderers     1144.503168
Xerez Club Deportivo        1013.116122
Évian Thonon Gaillard FC    1089.739249
Length: 254, dtype: float64